# Load data via the API


Before running this notebook, initialize the database:

```
docker-compose exec web flask db init
```

In [1]:
import requests
import json
import os

In [2]:
BASE_URL = "http://web:8000/api/v0.0"
data_dir = os.getenv("DATA_DIR")

## Query for constants

In [3]:
response = requests.get(f'{BASE_URL}/entity_types')
response.raise_for_status()
response.json()

[['Radar', 'MeteoRadarMosaic', None],
 ['Station', 'PointWeatherObserver', None],
 ['Station', 'TemperatureMosaic', None],
 ['Station', 'EnergyConsumptionMonitor', None]]

## Load sources

In [4]:
with open(os.path.join(data_dir, 'sources.json')) as f:
    data = json.load(f)['sources']

In [5]:
response = requests.post(f'{BASE_URL}/sources', json=data)
response.raise_for_status()
tdmq_ids = response.json()

In [6]:
tdmq_ids

['e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6',
 '2bb83248-27f4-53c9-b149-ca74c8c283ee',
 '42608b02-a82a-58ae-84fb-f358bf9b7494',
 '8b53505d-79b8-548d-b160-777d116cb6b2',
 '5df07546-b1d6-517f-8798-5abd79e7ebb1']

In [7]:
response = requests.get(f'{BASE_URL}/sources')
response.raise_for_status()
sources = response.json()

In [8]:
sources

[{'default_footprint': {'coordinates': [9.22100000642642, 30.0000000019687],
   'type': 'Point'},
  'entity_category': 'Station',
  'entity_type': 'PointWeatherObserver',
  'external_id': 'tdm/sensor_0',
  'tdmq_id': 'e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6'},
 {'default_footprint': {'coordinates': [9.22200000642623, 30.0030000019686],
   'type': 'Point'},
  'entity_category': 'Station',
  'entity_type': 'PointWeatherObserver',
  'external_id': 'tdm/sensor_1',
  'tdmq_id': '2bb83248-27f4-53c9-b149-ca74c8c283ee'},
 {'default_footprint': {'coordinates': [9.55100000643269, 32.0000000018894],
   'type': 'Point'},
  'entity_category': 'Station',
  'entity_type': 'EnergyConsumptionMonitor',
  'external_id': 'tdm/sensor_3',
  'tdmq_id': '42608b02-a82a-58ae-84fb-f358bf9b7494'},
 {'default_footprint': {'coordinates': [9.34300000645912, 31.2000000019276],
   'type': 'Point'},
  'entity_category': 'Station',
  'entity_type': 'PointWeatherObserver',
  'external_id': 'tdm/sensor_4',
  'tdmq_id': '8b53

In [9]:
for s in sources:
    response = requests.get(f'{BASE_URL}/sources/{s["tdmq_id"]}')
    response.raise_for_status()
    print('*'*20)
    print(response.json())                       

********************
{'default_footprint': {'coordinates': [9.22100000642642, 30.0000000019687], 'type': 'Point'}, 'description': {'alias': 'my desk', 'controlledProperties': ['temperature', 'humidity'], 'default_footprint': {'coordinates': [9.221, 30.0], 'type': 'Point'}, 'description': {'brandName': 'ProSensor', 'entity_type': 'multisensor', 'manufacturerName': 'CRS4', 'modelName': 'R2D2'}, 'entity_category': 'Station', 'entity_type': 'PointWeatherObserver', 'id': 'tdm/sensor_0', 'stationary': True}, 'entity_category': 'Station', 'entity_type': 'PointWeatherObserver', 'external_id': 'tdm/sensor_0', 'stationary': True, 'tdmq_id': 'e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6'}
********************
{'default_footprint': {'coordinates': [9.22200000642623, 30.0030000019686], 'type': 'Point'}, 'description': {'alias': 'B221', 'controlledProperties': ['humidity', 'temperature'], 'default_footprint': {'coordinates': [9.222, 30.003], 'type': 'Point'}, 'description': {'brandName': 'ProSensor', 'entit

## Load records

In [10]:
with open(os.path.join(data_dir, 'records.json')) as f:
    data = json.load(f)['records']

In [11]:
response = requests.post(f'{BASE_URL}/records', json=data)
response.raise_for_status()
tdmq_ids = response.json()

## Run some queries

In [12]:
def query(url):
    response = requests.get(f'{BASE_URL}{url}')
    response.raise_for_status()
    print(json.dumps(response.json(), indent=4))

In [13]:
query('/sources?id=tdm/sensor_3')

[
    {
        "default_footprint": {
            "coordinates": [
                9.55100000643269,
                32.0000000018894
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type": "EnergyConsumptionMonitor",
        "external_id": "tdm/sensor_3",
        "tdmq_id": "42608b02-a82a-58ae-84fb-f358bf9b7494"
    }
]


In [15]:
query('/sources?entity_category=Station')

[
    {
        "default_footprint": {
            "coordinates": [
                9.22100000642642,
                30.0000000019687
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_0",
        "tdmq_id": "e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6"
    },
    {
        "default_footprint": {
            "coordinates": [
                9.22200000642623,
                30.0030000019686
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_1",
        "tdmq_id": "2bb83248-27f4-53c9-b149-ca74c8c283ee"
    },
    {
        "default_footprint": {
            "coordinates": [
                9.55100000643269,
                32.0000000018894
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type"

In [16]:
query('/sources?entity_type=EnergyConsumptionMonitor')

[
    {
        "default_footprint": {
            "coordinates": [
                9.55100000643269,
                32.0000000018894
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type": "EnergyConsumptionMonitor",
        "external_id": "tdm/sensor_3",
        "tdmq_id": "42608b02-a82a-58ae-84fb-f358bf9b7494"
    }
]


In [17]:
query('/sources?controlledProperties=temperature')

[
    {
        "default_footprint": {
            "coordinates": [
                9.22100000642642,
                30.0000000019687
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_0",
        "tdmq_id": "e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6"
    },
    {
        "default_footprint": {
            "coordinates": [
                9.22200000642623,
                30.0030000019686
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_1",
        "tdmq_id": "2bb83248-27f4-53c9-b149-ca74c8c283ee"
    },
    {
        "default_footprint": {
            "coordinates": [
                9.34300000645912,
                31.2000000019276
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type"

In [18]:
query('/sources?controlledProperties=temperature,humidity')

[
    {
        "default_footprint": {
            "coordinates": [
                9.22100000642642,
                30.0000000019687
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_0",
        "tdmq_id": "e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6"
    },
    {
        "default_footprint": {
            "coordinates": [
                9.22200000642623,
                30.0030000019686
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_1",
        "tdmq_id": "2bb83248-27f4-53c9-b149-ca74c8c283ee"
    }
]


In [19]:
query('/sources?after=2019-05-02T11:00:20Z')

[
    {
        "default_footprint": {
            "coordinates": [
                9.22100000642642,
                30.0000000019687
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_0",
        "tdmq_id": "e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6"
    }
]


In [20]:
query('/sources?stationary=false')

[
    {
        "default_footprint": {
            "coordinates": [
                9.11900000640193,
                29.0000000019969
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_5",
        "tdmq_id": "5df07546-b1d6-517f-8798-5abd79e7ebb1"
    }
]


In [21]:
query('/sources?stationary=true')

[
    {
        "default_footprint": {
            "coordinates": [
                9.22100000642642,
                30.0000000019687
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_0",
        "tdmq_id": "e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6"
    },
    {
        "default_footprint": {
            "coordinates": [
                9.22200000642623,
                30.0030000019686
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_1",
        "tdmq_id": "2bb83248-27f4-53c9-b149-ca74c8c283ee"
    },
    {
        "default_footprint": {
            "coordinates": [
                9.55100000643269,
                32.0000000018894
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type"

In [22]:
query('/sources?brandName=Acme')

[
    {
        "default_footprint": {
            "coordinates": [
                9.34300000645912,
                31.2000000019276
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_4",
        "tdmq_id": "8b53505d-79b8-548d-b160-777d116cb6b2"
    },
    {
        "default_footprint": {
            "coordinates": [
                9.11900000640193,
                29.0000000019969
            ],
            "type": "Point"
        },
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_5",
        "tdmq_id": "5df07546-b1d6-517f-8798-5abd79e7ebb1"
    }
]


In [23]:
for s in sources:
    response = requests.delete(f'{BASE_URL}/sources/{s["tdmq_id"]}')
    response.raise_for_status()

In [24]:
response = requests.get(f'{BASE_URL}/sources')
response.raise_for_status()
response.json()

[]